# Waifu Diffusion

https://huggingface.co/hakurei/waifu-diffusion

In [ ]:
try:
    #from diffusers import StableDiffusionPipeline
    import diffusers
except:
    !pip install -q transformers
    !pip install -q accelerate
    !pip install -q diffusers
    #from diffusers import StableDiffusionPipeline
    #from diffusers.pipelines.stable_diffusion import safety_checker
    import diffusers

import torch
import matplotlib.pyplot as plt
import numpy as np
import time

if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device("cpu")

print("Using {}.".format(device_name))

In [ ]:
pipe = diffusers.StableDiffusionPipeline.from_pretrained("hakurei/waifu-diffusion", torch_dtype = torch.float32)
pipe = pipe.to(device_name)
pipe.scheduler = diffusers.EulerDiscreteScheduler.from_config(pipe.scheduler.config)
pipe.safety_checker = lambda images, **kwargs: [images, [False] * len(images)]
pipe = pipe.to(device_name)

In [ ]:
steps = 50
width = 512
height = 512
scale = 7.5
seed = 23904253959
N = 1

gen = torch.Generator(device = device_name).manual_seed(seed)

prompt = "(masterpiece),(best quality),(8k wallpaper),(beautiful detailed face),"
prompt += "perfect anatomy,(1girl),solo,long hair,(white bikini),(large breasts),spreading legs,"
prompt += "seductive expression,slim,thin waist,mature woman,mature face,(cleavage)"

negative_prompt = "deformed,blurry,bad anatomy,disfigured,poorly drawn face,mutation,mutated,extra limb,ugly,poorly drawn hands,two heads,child,"
negative_prompt += "kid,gross,mutilated,disgusting,horrible,scary,evil,old,conjoined"

image_list = []
with torch.autocast("cuda"): 
    for i in range(N): 
        image = pipe(prompt, height = height, width = width, num_inference_steps = steps, guidance_scale = scale,
                     negative_prompt = negative_prompt, generator = gen)
        image_list.append(image)

In [ ]:
for i in range(len(image_list)):
    display(image_list[i].images[0])